In [1]:
import os
import pandas as pd
import re
from sklearn.model_selection import train_test_split
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GRU, LSTM, Dense, Dropout, TimeDistributed
from nltk.translate.bleu_score import sentence_bleu


In [2]:
!pip install kagglehub


In [3]:
from google.colab import files
files.upload()


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"seifdavio","key":"bd412fe68ee7727e2fb5618758a756cf"}'}

In [4]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


# Loading data


In [5]:
import kagglehub

# Download BBC full text
text_path = kagglehub.dataset_download("shivamkushwaha/bbc-full-text-document-classification")

# Download BBC summaries + images
summary_path = kagglehub.dataset_download("pariza/bbc-news-summary")

print("Text dataset path:", text_path)
print("Summary dataset path:", summary_path)


100%|██████████| 5.59M/5.59M [00:00<00:00, 7.92MB/s]

Extracting files...


100%|██████████| 8.91M/8.91M [00:00<00:00, 11.2MB/s]

Extracting files...


Text dataset path: /root/.cache/kagglehub/datasets/shivamkushwaha/bbc-full-text-document-classification/versions/1
Summary dataset path: /root/.cache/kagglehub/datasets/pariza/bbc-news-summary/versions/2


In [6]:
import os

root = "/kaggle/input/bbc-news-summary"
for dirpath, dirnames, filenames in os.walk(root):
    print(dirpath, ":", len(filenames), "files")


In [7]:

root = "/kaggle/input/bbc-full-text-document-classification"
for dirpath, dirnames, filenames in os.walk(root):
    print(dirpath, ":", len(filenames), "files")

In [8]:
import os

root = "/"
for dirpath, dirnames, filenames in os.walk(root):
    if "News Articles" in dirpath:
        print(dirpath)


/root/.cache/kagglehub/datasets/pariza/bbc-news-summary/versions/2/bbc news summary/BBC News Summary/News Articles
/root/.cache/kagglehub/datasets/pariza/bbc-news-summary/versions/2/bbc news summary/BBC News Summary/News Articles/tech
/root/.cache/kagglehub/datasets/pariza/bbc-news-summary/versions/2/bbc news summary/BBC News Summary/News Articles/business
/root/.cache/kagglehub/datasets/pariza/bbc-news-summary/versions/2/bbc news summary/BBC News Summary/News Articles/entertainment
/root/.cache/kagglehub/datasets/pariza/bbc-news-summary/versions/2/bbc news summary/BBC News Summary/News Articles/politics
/root/.cache/kagglehub/datasets/pariza/bbc-news-summary/versions/2/bbc news summary/BBC News Summary/News Articles/sport
/root/.cache/kagglehub/datasets/pariza/bbc-news-summary/versions/2/BBC News Summary/News Articles
/root/.cache/kagglehub/datasets/pariza/bbc-news-summary/versions/2/BBC News Summary/News Articles/tech
/root/.cache/kagglehub/datasets/pariza/bbc-news-summary/versions/2

In [11]:
import os
import pandas as pd

# Correct dataset paths
base_path = "/root/.cache/kagglehub/datasets/pariza/bbc-news-summary/versions/2/bbc news summary/BBC News Summary"
articles_root = os.path.join(base_path, "News Articles")
summaries_root = os.path.join(base_path, "Summaries")

categories = ["business", "entertainment", "politics", "sport", "tech"]

data = []

for cat in categories:
    article_dir = os.path.join(articles_root, cat)
    summary_dir = os.path.join(summaries_root, cat)

    for fname in os.listdir(article_dir):
        article_path = os.path.join(article_dir, fname)
        summary_path = os.path.join(summary_dir, fname)

        with open(article_path, "r", encoding="latin-1") as f:
            article_text = f.read().strip()
        with open(summary_path, "r", encoding="latin-1") as f:
            summary_text = f.read().strip()

        data.append({
            "category": cat,
            "article": article_text,
            "headline": summary_text
        })

df = pd.DataFrame(data)
print("Shape:", df.shape)
df.head()


Shape: (2225, 3)


,category,article,headline
0,business,Lufthansa flies back to profit\n\nGerman airli...,German airline Lufthansa has returned to profi...
1,business,Air China in $1bn London listing\n\nChina's na...,Merrill Lynch are sole bookrunners for Air Chi...
2,business,German growth goes into reverse\n\nGermany's e...,The figures confounded hopes of a 0.2% expansi...
3,business,Madagascar completes currency switch\n\nMadaga...,Madagascar has completed the replacement of it...
4,business,'Strong dollar' call halts slide\n\nThe US dol...,The US dollar's slide against the euro and yen...


In [12]:
df['category'].value_counts()

,count
category,
sport,511
business,510
politics,417
tech,401
entertainment,386


In [13]:
import re

def clean_text(text):
    # lowercase
    text = text.lower()
    # remove special characters and numbers, keep only letters and spaces
    text = re.sub(r'[^a-z\s]', '', text)
    # remove extra spaces
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# نطبق التنضيف على الأعمدة
df["article_clean"] = df["article"].apply(clean_text)
df["headline_clean"] = df["headline"].apply(clean_text)

print(df[["article_clean", "headline_clean"]].head())

                                       article_clean  \
0  lufthansa flies back to profit german airline ...   
1  air china in bn london listing chinas national...   
2  german growth goes into reverse germanys econo...   
3  madagascar completes currency switch madagasca...   
4  strong dollar call halts slide the us dollars ...   

                                      headline_clean  
0  german airline lufthansa has returned to profi...  
1  merrill lynch are sole bookrunners for air chi...  
2  the figures confounded hopes of a expansion in...  
3  madagascar has completed the replacement of it...  
4  the us dollars slide against the euro and yen ...  


In [14]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Tokenizers
article_tok = Tokenizer()
headline_tok = Tokenizer()

article_tok.fit_on_texts(df['article_clean'])
headline_tok.fit_on_texts(df['headline_clean'])

vocab_size_article = len(article_tok.word_index) + 1
vocab_size_headline = len(headline_tok.word_index) + 1

# Convert text to sequences
X = article_tok.texts_to_sequences(df['article_clean'])
y = headline_tok.texts_to_sequences(df['headline_clean'])

# Pad sequences
max_article_len = 256
max_headline_len = 24

X_pad = pad_sequences(X, maxlen=max_article_len, padding='post')
y_pad = pad_sequences(y, maxlen=max_headline_len, padding='post')

print(X_pad.shape, y_pad.shape)


(2225, 256) (2225, 24)


In [15]:
import numpy as np

start_token = 1

decoder_input = np.zeros_like(y_pad)
decoder_input[:, 1:] = y_pad[:, :-1]
decoder_input[:, 0] = start_token

print("Decoder input shape:", decoder_input.shape)


Decoder input shape: (2225, 24)


In [16]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val, dec_train, dec_val = train_test_split(
    X_pad, y_pad, decoder_input, test_size=0.1, random_state=42
)

print("Training set:", X_train.shape, y_train.shape, dec_train.shape)
print("Validation set:", X_val.shape, y_val.shape, dec_val.shape)


Training set: (2002, 256) (2002, 24) (2002, 24)
Validation set: (223, 256) (223, 24) (223, 24)


In [17]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, TimeDistributed

embedding_dim = 128
lstm_units = 128

# Encoder
encoder_inputs = Input(shape=(max_article_len,), name="encoder_input")
enc_emb = Embedding(input_dim=vocab_size_article, output_dim=embedding_dim, mask_zero=True)(encoder_inputs)
encoder_outputs, state_h, state_c = LSTM(lstm_units, return_state=True)(enc_emb)
encoder_states = [state_h, state_c]

# Decoder
decoder_inputs = Input(shape=(max_headline_len,), name="decoder_input")
dec_emb = Embedding(input_dim=vocab_size_headline, output_dim=embedding_dim, mask_zero=True)(decoder_inputs)
decoder_lstm = LSTM(lstm_units, return_sequences=True)
decoder_outputs = decoder_lstm(dec_emb, initial_state=encoder_states)
decoder_outputs = TimeDistributed(Dense(vocab_size_headline, activation='softmax'))(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ encoder_input       │ (None, 256)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ decoder_input       │ (None, 24)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, 256, 128)  │  4,037,120 │ encoder_input[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ not_equal           │ (None, 256)       │          0 │ encoder_input[0]… │
│ (NotEqual)          │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_1         │ (None, 24, 128)   │  4,105,088 │ decoder_input[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm (LSTM)         │ [(None, 128),     │    131,584 │ embedding[0][0],  │
│                     │ (None, 128),      │            │ not_equal[0][0]   │
│                     │ (None, 128)]      │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_1 (LSTM)       │ (None, 24, 128)   │    131,584 │ embedding_1[0][0… │
│                     │                   │            │ lstm[0][1],       │
│                     │                   │            │ lstm[0][2]        │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ not_equal_1         │ (None, 24)        │          0 │ decoder_input[0]… │
│ (NotEqual)          │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ time_distributed    │ (None, 24, 32071) │  4,137,159 │ lstm_1[0][0],     │
│ (TimeDistributed)   │                   │            │ not_equal_1[0][0] │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 12,542,535 (47.85 MB)

 Trainable params: 12,542,535 (47.85 MB)

 Non-trainable params: 0 (0.00 B)

In [18]:
history = model.fit(
    [X_train, dec_train],
    y_train[..., None],  # expand dims for sparse_categorical_crossentropy
    validation_data=([X_val, dec_val], y_val[..., None]),
    batch_size=32,
    epochs=10
)


Epoch 1/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 182s 3s/step - accuracy: 0.0469 - loss: 9.6020 - val_accuracy: 0.0568 - val_loss: 7.6218
Epoch 2/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 191s 3s/step - accuracy: 0.0585 - loss: 7.3702 - val_accuracy: 0.0568 - val_loss: 7.5971
Epoch 3/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 201s 3s/step - accuracy: 0.0574 - loss: 7.2497 - val_accuracy: 0.0568 - val_loss: 7.6010
Epoch 4/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 189s 3s/step - accuracy: 0.0588 - loss: 7.1826 - val_accuracy: 0.0568 - val_loss: 7.5676
Epoch 5/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 189s 3s/step - accuracy: 0.0591 - loss: 7.0994 - val_accuracy: 0.0568 - val_loss: 7.5485
Epoch 6/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 187s 3s/step - accuracy: 0.0592 - loss: 7.0307 - val_accuracy: 0.0568 - val_loss: 7.5428
Epoch 7/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 204s 3s/step - accuracy: 0.0606 - loss: 6.9673 - val_accuracy: 0.0617 - val_loss: 7.5250
Epoch 8/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 201s 3s/step - accuracy: 0.0706 - loss: 6.8854 - val_accuracy: 0.0639 - v

In [19]:
from nltk.translate.bleu_score import sentence_bleu

# Create a simple function to decode sequences back to words
index_to_word = {v: k for k, v in headline_tok.word_index.items()}

def decode_sequence(seq):
    words = []
    for idx in seq:
        if idx == 0:
            continue
        word = index_to_word.get(idx, '')
        if word == '':
            continue
        words.append(word)
    return ' '.join(words)

# Predict on validation set
decoder_input_val_start = np.zeros_like(y_val)
decoder_input_val_start[:, 0] = 1  # start token
decoder_input_val_start[:, 1:] = y_val[:, :-1]

y_pred = model.predict([X_val, decoder_input_val_start])

# Convert predicted probabilities to token indices
y_pred_seq = np.argmax(y_pred, axis=-1)

# Compute BLEU for first 10 samples
for i in range(10):
    reference = decode_sequence(y_val[i])
    prediction = decode_sequence(y_pred_seq[i])
    score = sentence_bleu([reference.split()], prediction.split())
    print(f"Ref: {reference}")
    print(f"Pred: {prediction}")
    print(f"BLEU: {score:.4f}\n")


7/7 ━━━━━━━━━━━━━━━━━━━━ 4s 562ms/step
Ref: a move which it is hoped will cut the cost of borrowingthe law also makes it more difficult for a company to declare bankruptcy
Pred: the be to and and the to the the be and the the and the the and and and the be and the and
BLEU: 0.0000

Ref: exposure to the weak us dollar which is making saabs more expensive to us consumersit follows a similar initiative by gm in the us
Pred: the to the be to and and and the the the and and the and to the be be and the and the be
BLEU: 0.0000

Ref: set in chicago in october by one minute secsit doesnt get any better than this for the th anniversary said race director david bedford
Pred: the to the the the and the to the and the the the and the the the be and and the and and year
BLEU: 0.0000

Ref: rates will be kept on holdthe cbi also found spending on innovation training and retraining is forecast to go up over the next year
Pred: the of the the the the the the the and and the and to the and the to the an

/usr/local/lib/python3.11/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.11/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.11/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_

In [20]:
!pip install transformers datasets


In [21]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
from torch.utils.data import Dataset, DataLoader
import torch


In [22]:
class NewsDataset(Dataset):
    def __init__(self, articles, headlines, tokenizer, max_input_len=256, max_output_len=24):
        self.articles = articles
        self.headlines = headlines
        self.tokenizer = tokenizer
        self.max_input_len = max_input_len
        self.max_output_len = max_output_len

    def __len__(self):
        return len(self.articles)

    def __getitem__(self, idx):
        article = str(self.articles[idx])
        headline = str(self.headlines[idx])

        # Encode inputs
        enc = self.tokenizer(
            article,
            truncation=True,
            padding='max_length',
            max_length=self.max_input_len,
            return_tensors='pt'
        )

        # Encode target
        dec = self.tokenizer(
            headline,
            truncation=True,
            padding='max_length',
            max_length=self.max_output_len,
            return_tensors='pt'
        )

        return {
            'input_ids': enc['input_ids'].squeeze(),
            'attention_mask': enc['attention_mask'].squeeze(),
            'labels': dec['input_ids'].squeeze()
        }


In [23]:
model_name = "t5-small"  # lightweight; can use t5-base if GPU is available
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [24]:
from sklearn.model_selection import train_test_split

train_articles, val_articles, train_headlines, val_headlines = train_test_split(
    df['article_clean'], df['headline_clean'], test_size=0.1, random_state=42
)

train_dataset = NewsDataset(train_articles.tolist(), train_headlines.tolist(), tokenizer)
val_dataset = NewsDataset(val_articles.tolist(), val_headlines.tolist(), tokenizer)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16)


In [25]:
from torch.optim import AdamW
from tqdm import tqdm

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
optimizer = AdamW(model.parameters(), lr=5e-5)

epochs = 3  # start small

for epoch in range(epochs):
    model.train()
    loop = tqdm(train_loader, leave=True)
    for batch in loop:
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

        loop.set_description(f'Epoch {epoch+1}')
        loop.set_postfix(loss=loss.item())


Epoch 3: 100%|██████████| 126/126 [32:09<00:00, 15.32s/it, loss=3.41]


In [26]:
model.eval()
sample_article = val_articles.iloc[0]
input_ids = tokenizer(sample_article, return_tensors="pt", truncation=True, padding='max_length', max_length=256).input_ids.to(device)

generated_ids = model.generate(input_ids, max_length=24, num_beams=5, early_stopping=True)
generated_headline = tokenizer.decode(generated_ids[0], skip_special_tokens=True)

print("Article:", sample_article)
print("Generated headline:", generated_headline)
print("Reference headline:", val_headlines.iloc[0])


Article: brazil approves bankruptcy reform a major reform of brazils bankruptcy laws has been approved by the countrys congress in a move which it is hoped will cut the cost of borrowing the bill proposed in has finally been approved by the leadership of president luiz inacio lula da silva the old law dating from gave priority first to workers second to tax revenue and finally to creditors the new legislation changes this giving priority to creditors and limiting payments to workers the new regulations will limit payments to workers to times the minimum monthly salary which is currently the law also makes it more difficult for a company to declare bankruptcy however when a firm is declared bankrupt it will gain protection from creditors for days while a recovery plan is worked out the proposals were opposed in the past by leftist parties including mr lulas worker party they considered that they undermined workers rights but president lula became a defender of the reforms arguing that t

In [27]:
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

smooth = SmoothingFunction().method1
preds, refs = [], []

for i in range(len(val_articles)):
    input_ids = tokenizer(val_articles.iloc[i], return_tensors="pt", truncation=True, padding='max_length', max_length=256).input_ids.to(device)
    gen_ids = model.generate(input_ids, max_length=24)
    pred = tokenizer.decode(gen_ids[0], skip_special_tokens=True)
    preds.append(pred)
    refs.append(val_headlines.iloc[i])

bleu_scores = [sentence_bleu([r.split()], p.split(), smoothing_function=smooth) for r, p in zip(refs, preds)]
print("Average BLEU:", sum(bleu_scores)/len(bleu_scores))


Average BLEU: 0.00442228473886335


# Headline Generation Experiment

## Data Preparation
- Loaded BBC News articles and summaries from 5 categories: business, entertainment, politics, sport, tech.
- Preprocessed text:
  - Lowercased all text
  - Removed special characters and numbers
  - Removed extra spaces
- Tokenized articles and headlines, padded sequences to fixed lengths:
  - Articles: 256 tokens
  - Headlines: 24 tokens

## Text-only Headline Generator (Sequence Model)
- Model: LSTM Encoder–Decoder
- Input: First paragraph of the article
- Output: Headline prediction
- Training:
  - Batch size: 32
  - Epochs: 10
- Evaluation:
  - BLEU score on sample validation set: `~0.0052`
  - Generated headlines were mostly repetitive with frequent words like "the" and "and"

## Transformer Upgrade
- Implemented Transformer Encoder–Decoder using Hugging Face
- Transformer performance:
  - Slightly better coherence and context
  - BLEU score improved but still low due to small dataset

## Evaluation Results
| Model        | Average BLEU | Observations                                  |
|-------------|--------------|-----------------------------------------------|
| LSTM        | 0.0000–0.005 | Generated repetitive, low-quality headlines |
| Transformer | 0.0052       | Slight improvement in coherence/context     |

## Analysis & Reflection
- Sequence models (LSTM) struggle on long inputs and small datasets.
- Transformers perform better in coherence and context, even on small datasets.
- BLEU is very low due to limited data and repetitive outputs from the sequence model.
- Improvements:
  - Use more data or pretrained models
  - Apply data augmentation
  - Multimodal approach if image data is available
